# ANALYZING CIA FACTBOOK DATA

we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
- population — the global population.
- population_growth — the annual population growth rate, as a percentage.
- area — the total land and water area.


In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## OVERVIEW OF THE DATA

In [2]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


### SUMMARY

In [3]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- name — the name of the country.
- area— the country's total area (both land and water).
- area_land — the country's land area in square kilometers.
- area_water — the country's water area in square kilometers.
- population — the country's population.
- population_growth— the country's population growth as a percentage.
- birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
- death_rate — the country's death rate, or the number of death per year per 1,000 people.


### CHECKING FOR OUTLIERS

In [4]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
    FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


In [5]:
%%sql
SELECT name, MIN(population)
    FROM facts

Done.


name,MIN(population)
Antarctica,0


In [6]:
%%sql
SELECT name, MAX(population)
    FROM facts

Done.


name,MAX(population)
World,7256490011


It seems like the table contains a row for the whole *world*, which explains the population of over 7.2 billion. It also seems like the table contains a row for *Antarctica*, which explains the population of 0.

Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


### FINDING DENSELY POPULATED COUNTRIES

In [8]:
%%sql
SELECT AVG(population) AS mean_pop, AVG(area) AS mean_area
    FROM facts
    WHERE name <> "World"

Done.


mean_pop,mean_area
32242666.56846473,555093.546184739


We'll identify countries that have the following:

    Above-average values for population.
    Below-average values for area.


In [9]:
%%sql
SELECT name, population, area
    FROM facts
    WHERE population > (SELECT AVG(population)
    FROM facts WHERE name <> "World") AND area < (SELECT AVG(area)
    FROM facts WHERE name <> "World")

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


Country with most people

In [10]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population) FROM FACTS WHERE name <> "World")

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


Country with most population growth

In [11]:
%%sql
SELECT *
    FROM facts
    WHERE population_growth == (SELECT MAX(population_growth) FROM facts WHERE name <> "World")

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


Countries with highest land to water ratio

In [12]:
%%sql 
SELECT *, area_land/area_water AS land_water_ratio
    FROM facts
    ORDER BY land_water_ratio DESC
    LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,land_water_ratio
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38,5118
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56,4222
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36,1785
70,gv,Guinea,245857,245717,140,11780162,2.63,35.74,9.46,0.0,1755
42,cs,Costa Rica,51100,51060,40,4814144,1.22,15.91,4.55,0.83,1276


- Countries with highest water to land ratio

In [13]:
%%sql 
SELECT *, area_water/area_land AS water_land_ratio
    FROM facts
    ORDER BY water_land_ratio DESC
    LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_land_ratio
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51,0
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3,0
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92,0


In [14]:
%%sql 
SELECT *, area_land/area_water AS land_water_ratio
    FROM facts
    WHERE land_water_ratio < 1
    ORDER BY land_water_ratio 

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,land_water_ratio
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,0
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,0


Countries that will add the most people to their population next year. In this case we look for the country with the highest birth rate

In [20]:
%%sql
SELECT name, population_growth, birth_rate
    FROM facts
    ORDER BY birth_rate DESC
    LIMIT 10

Done.


name,population_growth,birth_rate
Niger,3.25,45.45
Mali,2.98,44.99
Uganda,3.24,43.79
Zambia,2.88,42.13
Burkina Faso,3.03,42.03
Burundi,3.28,42.01
Malawi,3.32,41.56
Somalia,1.83,40.45
Angola,2.78,38.78
Mozambique,2.45,38.58


From the result we can see that most of them are African countries

- Countries with high death rate to birth rate ratio

In [16]:
%%sql 
SELECT *, ROUND(death_rate/birth_rate, 4) AS death_birth_ratio
    FROM facts
    ORDER BY death_birth_ratio DESC
    LIMIT 10

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,death_birth_ratio
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29,1.6188
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0,1.5044
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26,1.431
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27,1.4129
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33,1.3897
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,1.3895
158,si,Slovenia,20273,20151,122,1983412,0.26,8.42,11.37,0.37,1.3504
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25,1.3489
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24,1.3483
214,sb,Saint Pierre and Miquelon,242,242,0,5657,1.08,7.42,9.72,8.49,1.31


Most of the top countries here are europeans

- Countries with highest population per area

In [17]:
%%sql
SELECT *, population/area AS pop_per_area
    FROM facts
    ORDER BY pop_per_area DESC
    LIMIT 10

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,pop_per_area
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8141
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6445
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299
